In [54]:
import numpy as np
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import sys 
from datetime import datetime
from selenium import webdriver


In [55]:
df =pd.read_csv("1227_coinurl_final2.csv")
df.head()

,no,Coin_Name,Coin_Abb,URL
0,1,Bitcoin,BTC,https://coinmarketcap.com/currencies/bitcoin/
1,2,XRP,XRP,https://coinmarketcap.com/currencies/ripple/
2,3,Ethereum,ETH,https://coinmarketcap.com/currencies/ethereum/
3,4,Bitcoin Cash,BCH,https://coinmarketcap.com/currencies/bitcoin-c...
4,5,EOS,EOS,https://coinmarketcap.com/currencies/eos/


In [56]:
url_base =df['URL']
url_add = 'historical-data/?start=20130428&&end=20181226'#coinmarketcap의 모든 코인 스타팅 값 130428
url = url_base+url_add
url[0]

'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&&end=20181226'

In [38]:
df['Coin_Name'][0]

'Bitcoin'

In [4]:
import requests 
url2 = 'https://httpbin.org/user-agent'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get(url2,headers=headers)
html = response.content
print(response.content)

USER_AGENTS = [
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/57.0.2987.110 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/61.0.3163.79 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
     'Gecko/20100101 '
     'Firefox/55.0'),  # firefox
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/61.0.3163.91 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/62.0.3202.89 '
     'Safari/537.36'),  # chrome
    ('Mozilla/5.0 (X11; Linux x86_64) '
     'AppleWebKit/537.36 (KHTML, like Gecko) '
     'Chrome/63.0.3239.108 '
     'Safari/537.36'),  # chrome
]

b'{\n  "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36"\n}\n'


In [ ]:
import time
import requests 
from tqdm import tqdm_notebook

data = []
i = 0;


for n in tqdm_notebook(df.index):
    
    
    html = urlopen(url[n])
    soup_tmp = BeautifulSoup(html, 'html.parser')
    priceDiv = soup_tmp.find('div', attrs={'class':'table-responsive'})
    rows = priceDiv.find_all('tr')
    coinName = soup_tmp.find('h2').get_text()
    
    


    for row in rows:
        tmp = []
        tds = row.findChildren()

        for td in tds:
            tmp.append(td.text)

        if(i > 0):
            tmp[0] = tmp[0].replace(',','')
            tmp[5] = tmp[5].replace(',','')
            tmp[6] = tmp[6].replace(',','')
            
            try:
                data.append({'no':(n+1),
                             'date':datetime.strptime(tmp[0], '%b %d %Y'),
                             'coinName':coinName,
                             'open':float(tmp[1]),
                             'high':float(tmp[2]),
                             'low':float(tmp[3]),
                             'close':float(tmp[4]),
                             'volume':float(tmp[5]),
                             'mcap':float(tmp[6])})
                 
            except ValueError: 
                pass
                
       
            

        i = i + 1;

    df = pd.DataFrame(data)
    df.to_csv('1228_coin_historical_data.csv', sep=',', encoding='UTF-8')
    time.sleep(5)